### Convolutional Neural Network for CIFAR-10

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt


from Cifar10 import Cifar10
import helpers
import random

In [9]:
# device configuration
# device = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

print(f'Using device {device}')

# consitency in training
torch.manual_seed(1234)
random.seed(1234)

Using device cuda


In [25]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = Cifar10(train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = Cifar10(train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [28]:
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class Net2(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layer = nn.Sequential(
             nn.Conv2d(3, 32, 3),
             nn.Conv2d(32, 64, 3),
             nn.MaxPool2d(2,2),
             nn.Conv2d(64, 128, 3),
             nn.MaxPool2d(2, 2),
             nn.Conv2d(128, 256, 3),
             nn.MaxPool2d(2,2),
        )
        
        self.ff_layer = nn.Sequential(
            nn.Linear(1024, 640),
            nn.ReLU(inplace=True),
            nn.Linear(640, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.ff_layer(x)
        return x

class Net3(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv_layer = nn.Sequential(
             nn.Conv2d(3, 16, 3),
             nn.Conv2d(16, 64, 3),
             nn.AvgPool2d(2,2),
             nn.Conv2d(64, 128, 3),
             nn.MaxPool2d(2, 2),
             nn.Conv2d(128, 256, 3),
             nn.MaxPool2d(2,2),
        )
        
        self.ff_layer = nn.Sequential(
            nn.Linear(1024, 640),
            nn.ReLU(inplace=True),
            nn.Linear(640, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.ff_layer(x)
        return x




In [29]:
num_epochs = 3

# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

param_options = [
    (optim.SGD, {'lr':0.001, 'momentum':0.9}, Net1, 'SGD Net1'),
    (optim.SGD, {'lr':0.001, 'momentum':0.9}, Net2, 'SGD Net2'),
    (optim.SGD, {'lr':0.001, 'momentum':0.9}, Net3, 'SGD Net3'),
    (optim.Adam, {'lr':0.001}, Net1, 'Adam Net1'),
    (optim.Adam, {'lr':0.001}, Net2, 'Adam Net2'),
    (optim.Adam, {'lr':0.001}, Net3, 'Adam Net3'),
]

metrics = []
for opt, kwargs, Net, label in param_options:
    net = Net().to(device)
    # loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = opt(net.parameters(), **kwargs)
    loss_vals, tr_acc, te_acc  = helpers.train_model(net, trainloader, testloader, device, criterion, optimizer, num_epochs)

    metrics.append((loss_vals, tr_acc, te_acc, label))

starting training
[1,   100] loss: 2.303 tr acc: 0.100 te acc: 0.104
[1,   200] loss: 2.302 tr acc: 0.100 te acc: 0.104
[1,   300] loss: 2.302 tr acc: 0.129 te acc: 0.128
[1,   400] loss: 2.301 tr acc: 0.121 te acc: 0.120
[2,   100] loss: 2.300 tr acc: 0.110 te acc: 0.107
[2,   200] loss: 2.299 tr acc: 0.131 te acc: 0.126
[2,   300] loss: 2.297 tr acc: 0.131 te acc: 0.125
[2,   400] loss: 2.294 tr acc: 0.156 te acc: 0.150


In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10,10))


for idx, (loss_vals, tr_acc, te_acc, label) in enumerate(metrics):
    # ax1, ax2 = axes[idx]
    ax = axes[idx%3][idx//3]
    # ax1.plot(range(len(loss_vals)), loss_vals, label=f'Loss {label}')
    ax.plot(range(len(loss_vals)), tr_acc, label=f'train acc {label}')
    ax.plot(range(len(loss_vals)), te_acc, label=f'test acc {label}')

    ax.set_title(label)
    ax.legend()
    # ax2.legend()

plt.tight_layout()
plt.show()

metrics